In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import keras
from keras import layers
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, GRU
from keras.layers import TimeDistributed, Bidirectional, Conv1D, SimpleRNN, Embedding, MaxPooling1D, Flatten, ConvLSTM2D
from keras.models import Sequential
import time
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [12]:
climate_hour_train = pd.read_csv("C:/Users/arito/Documents/DePaul/CSC578/Final Project/data/climate_hour_train.csv")
climate_Xtest = pd.read_csv("C:/Users/arito/Documents/DePaul/CSC578/Final Project/data/climate_Xtest.csv", header=None)

In [13]:
values = climate_hour_train.values
groups = range(0, 14)
i = 1
plt.figure(figsize=(10,10))
for group in groups:
    plt.subplot(len(groups), 1, i)
    plt.plot(values[:, group])
    plt.title(climate_hour_train.columns[group], y=0.5, loc='right')
    i += 1
plt.show()

KeyboardInterrupt: 

In [3]:
climate_hour_train['p (mbar)'] = stats.zscore(climate_hour_train['p (mbar)'])
climate_hour_train['T (degC)'] = stats.zscore(climate_hour_train['T (degC)'])
climate_hour_train['Tpot (K)'] = stats.zscore(climate_hour_train['Tpot (K)'])
climate_hour_train['Tdew (degC)'] = stats.zscore(climate_hour_train['Tdew (degC)'])
climate_hour_train['rh (%)'] = stats.zscore(climate_hour_train['rh (%)'])
climate_hour_train['VPmax (mbar)'] = stats.zscore(climate_hour_train['VPmax (mbar)'])
climate_hour_train['VPdef (mbar)'] = stats.zscore(climate_hour_train['VPdef (mbar)'])
climate_hour_train['sh (g/kg)'] = stats.zscore(climate_hour_train['sh (g/kg)'])
climate_hour_train['H2OC (mmol/mol)'] = stats.zscore(climate_hour_train['H2OC (mmol/mol)'])
climate_hour_train['rho (g/m**3)'] = stats.zscore(climate_hour_train['rho (g/m**3)'])
climate_hour_train['wv (m/s)'] = stats.zscore(climate_hour_train['wv (m/s)'])
climate_hour_train['max. wv (m/s)'] = stats.zscore(climate_hour_train['max. wv (m/s)'])
climate_hour_train['wd (deg)'] = stats.zscore(climate_hour_train['wd (deg)'])

In [4]:
timestamp = climate_hour_train['Date Time']

In [5]:
climate_hour_train.drop('Date Time', axis=1, inplace=True)
climate_hour_train.head()

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,0.949502,-2.018375,-2.077203,-1.958654,1.092860,-1.324296,3.14,-0.785322,-1.506342,-1.509068,2.253761,-1.262182,-1.257560,0.217883
1,0.964153,-2.115644,-2.174821,-2.101663,1.019823,-1.352030,2.90,-0.781088,-1.562587,-1.565499,2.362318,-1.236053,-1.257560,0.190376
2,0.991013,-2.107441,-2.169011,-2.085774,1.038082,-1.350709,2.93,-0.783205,-1.555088,-1.558445,2.360587,-1.281780,-1.257560,-0.074379
3,1.009327,-2.135567,-2.198064,-2.137777,0.983305,-1.358633,2.85,-0.776853,-1.573836,-1.579607,2.395948,-1.334039,-1.365634,0.760002
4,1.066710,-2.203539,-2.270115,-2.228783,0.958959,-1.375802,2.71,-0.776853,-1.607584,-1.612525,2.484475,-1.138066,-1.149485,-0.191285


In [6]:
window_size = 23
cht = climate_hour_train.copy()
for i in range(window_size):
    climate_hour_train = pd.concat([climate_hour_train, cht.shift(-(i+1))], axis = 1)
climate_hour_train.dropna(axis=0, inplace=True)

In [7]:
climate_Xtest = stats.zscore(climate_Xtest)
climate_Xtest = pd.DataFrame(climate_Xtest)

In [8]:
x_train = climate_hour_train.iloc[:-1, :].values
y_train = climate_hour_train.iloc[1:,-13:-12].values
x_test = climate_Xtest.iloc[:, :].values

In [9]:
timesteps = 24
data_dim = 14
batch_size = 100
epochs = 30
x_train = x_train.reshape((x_train.shape[0], timesteps, data_dim))
x_test = x_test.reshape((x_test.shape[0], timesteps, data_dim))
print(x_train.shape, y_train.shape, x_test.shape)#, y_test.shape)

(52542, 24, 14) (52542, 1) (17447, 24, 14)


In [10]:
model = Sequential()
model.add(LSTM(50, dropout=0.2, batch_input_shape=(batch_size, timesteps, data_dim)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mae')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (100, 50)                 13000     
_________________________________________________________________
dense_1 (Dense)              (100, 1)                  51        
Total params: 13,051
Trainable params: 13,051
Non-trainable params: 0
_________________________________________________________________


In [11]:
start = time.time()
history = model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,shuffle=False)#, validation_data=(x_test,y_test)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
52400/52542 [============================>.] - ETA: 0s - loss: 0.2089

InvalidArgumentError: Incompatible shapes: [100] vs. [42]
	 [[{{node training/Adam/gradients/loss/dense_1_loss/mul_grad/Mul}}]]

In [2]:
test = pd.read_csv("C:/Users/arito/Documents/DePaul/CSC578/Final Project/data/climate_Xtest.csv", header=None)
train = pd.read_csv("C:/Users/arito/Documents/DePaul/CSC578/Final Project/data/climate_hour_train.csv")

In [3]:
train["Date Time"] = train["Date Time"].apply(lambda x: pd.datetime.strptime(x, '%d.%m.%Y %H:%M:%S'))
train["Date Time"] = pd.to_datetime(train["Date Time"])

In [4]:
xtrain = train.drop("Date Time", axis =1)

In [5]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i ==0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [8]:
values = xtrain.values
values = values.astype('float32')
scaler = StandardScaler()
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 24, 1)
data = reframed.drop(reframed.columns[np.arange(338,350)], axis=1)
data.drop(data.columns[[336]],axis=1, inplace=True)
X_train = data.values[:, :-1]
y_train = data.values[:,-1]
scalerZ = StandardScaler()
scaled_test = scalerZ.fit_transform(test)
train_X = X_train.reshape((X_train.shape[0], 24, 14))
y_train = y_train.reshape(-1,1)
test_x = scaled_test.reshape((scaled_test.shape[0], 24, 14))
print(train_X.shape, y_train.shape, test_x.shape)

(52542, 24, 14) (52542, 1) (17447, 24, 14)


In [19]:
X_train

array([[ 0.9495052 , -2.0183752 , -2.0772033 , ..., -1.1315333 ,
        -1.1494853 , -0.2142071 ],
       [ 0.96415555, -2.1156447 , -2.174821  , ..., -1.2164552 ,
        -1.0976096 , -0.29328987],
       [ 0.9910195 , -2.1074412 , -2.1690118 , ..., -1.0792736 ,
        -1.1494853 ,  0.41387117],
       ...,
       [ 1.921353  , -0.895675  , -1.0359452 , ..., -0.95515686,
        -0.80364764, -0.6233748 ],
       [ 1.8932669 , -0.90387845, -1.0417544 , ..., -0.9616893 ,
        -0.8555233 , -0.3253815 ],
       [ 1.9140278 , -0.9027065 , -1.0429176 , ..., -0.9355595 ,
        -0.751772  , -0.5053236 ]], dtype=float32)

In [10]:
X_train.shape

(52542, 336)

In [17]:
test_x

array([[[ 1.90341581e+00, -1.02505876e+00, -1.17392800e+00, ...,
          2.65162448e-04, -1.99789132e-03,  1.12422555e+00],
        [ 1.88840275e+00, -1.00496727e+00, -1.15289246e+00, ...,
          4.17564205e-04, -2.26210207e-03, -7.04182091e-01],
        [ 1.93102675e+00, -1.02731369e+00, -1.17856963e+00, ...,
          3.69626925e-03,  2.75549528e-03, -3.87481333e-01],
        ...,
        [ 1.55412943e+00, -1.04937485e+00, -1.16817714e+00, ...,
          1.26223363e-02,  1.64279218e-02, -5.09205679e-02],
        [ 1.50577034e+00, -1.06052566e+00, -1.17424128e+00, ...,
          1.26131144e-03,  1.96931554e-03, -4.91620114e-01],
        [ 1.36746387e+00, -1.08662167e+00, -1.18888043e+00, ...,
          2.07820107e-02,  2.23714948e-02, -2.47042174e-01]],

       [[ 1.88843350e+00, -1.00507785e+00, -1.15301957e+00, ...,
          4.17669098e-04, -2.26199611e-03, -7.04225103e-01],
        [ 1.93104417e+00, -1.02744434e+00, -1.17871870e+00, ...,
          3.69645718e-03,  2.75588886e

In [13]:
x_train = pd.DataFrame(X_train)

In [15]:
x_train.to_csv('x_train_work.csv')

In [11]:
y_train

array([[-1.5953118 ],
       [-1.5777328 ],
       [-1.5964836 ],
       ...,
       [-0.84528244],
       [-0.84762627],
       [-0.81950015]], dtype=float32)

In [12]:
y_train.shape

(52542, 1)

In [16]:
y_train = pd.DataFrame(y_train)
y_train.to_csv('y_train_work.csv')